In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from astropy.table import Table
from tqdm import tqdm
import healpy as hp
import treecorr
import DSS_functions as DSS_fct

In [2]:
from matplotlib import rc
rc('text', usetex=True)
rc('font', family='serif')


SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 15

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#### load all the necessary data, if the data is not preprocced before, I recommend to lower the resolution as very low scales are not considered anyway. Be aware that for the DSS foreground galaxies are needed, which are created with the corresponding density maps
#### pix are those pixels that belong to the octant

In [3]:
nside=2**11

load_prepared_data = True
if(load_prepared_data):
    density_contrast = np.load('data/density_contrast.npy')

    pix=np.load('data/pix_nside'+str(nside)+'.npy')
    gamma1=np.load('data/gamma1_nside'+str(nside)+'.npy')[pix]
    gamma2=np.load('data/gamma2_nside'+str(nside)+'.npy')[pix]

else:
    density_map = np.load('/global/homes/j/jharno/IA-infusion/SkySim5000/density/density_map_331_dens_allsky.npy')
    density_contrast_map = (density_map-density_map.mean())/density_map.mean()

    
    fn_map_curved = '/global/homes/j/jharno/IA-infusion/SkySim5000/kappa/1.0060kappa.npy'
    hpmap = np.load(fn_map_curved)
    hpmap_low=hp.pixelfunc.ud_grade(hpmap,nside_out=nside,power=-2)
    pix=np.where(hpmap_low!=-0)

    fn_gamma1="/global/homes/j/jharno/IA-infusion/SkySim5000/shear/1.0060gamma1.npy"
    fn_gamma1 = np.load(fn_gamma1)
    gamma1=hp.pixelfunc.ud_grade(fn_gamma1,nside_out=nside)[pix]
    
    fn_gamma2="/global/homes/j/jharno/IA-infusion/SkySim5000/shear/1.0060gamma2.npy"
    fn_gamma2 = np.load(fn_gamma2)
    gamma2=hp.pixelfunc.ud_grade(fn_gamma2,nside_out=nside)[pix]


#### Initialise the DSS class, specify here the galaxy bias and galaxy number density

In [4]:
DSS_class = DSS_fct.DSS_class(filter='top_hat',theta_ap=20,nside=nside)
Nap=DSS_class.calc_Nap(bias=1.5,n0=0.3,density_contrast=density_contrast)

density contrast does not have nisde 2048
SGD will be scaled to nside 2048


#### Create the tables that are needed or calculating the shear profiles, and then use again then DSS class.

In [7]:
ra,dec=hp.pix2ang(nside=nside,ipix=pix,lonlat=True)
Nap_table = Table(np.array([ra[0],dec[0],Nap[pix][0]]).T, names=('ra','dec','Nap'))
gamma_table = Table(np.array([ra[0],dec[0],-gamma1[0],gamma2[0]]).T, names=('ra','dec','gamma1','gamma2'))

shear_table=DSS_class.calc_shear(Nap_table=Nap_table,gamma_table=gamma_table,theta_min=5,theta_max=120,nbins=20,N_quantiles=5)

100%|██████████| 5/5 [03:58<00:00, 47.68s/it]


#### The datavector is then shear profiles + aperture number $N_\mathrm{ap}$. Save them to results

In [9]:
Nap_table.write('results/Nap_table_nside'+str(nside), format='fits',overwrite=True)
shear_table.write('results/shear_table_nside'+str(nside), format='fits',overwrite=True)